In [ ]:
import fastkaggle
import polars as pl

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate, KFold

In [ ]:
comp = "playground-series-s4e3"
path = fastkaggle.setup_comp(comp)

train = pl.read_csv(path/"train.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
test = pl.read_csv(path/"test.csv").drop("id").with_columns(pl.col(pl.Float64).cast(pl.Float32))
subm = pl.read_csv(path/"sample_submission.csv") 

In [ ]:
col_targets = train.columns[-7:]

In [ ]:
train.select(
    pl.reduce(function=lambda acc,x:acc+x, exprs=pl.col(col_targets)).alias("sum")>1
).sum()

In [ ]:
train_proc = (
    train.lazy()
    .filter(
        (pl.reduce(function=lambda acc,x:acc+x, exprs=pl.col(col_targets))<=1)
    )
    .with_columns(
        pl.concat_list(pl.col(col_targets)).list.arg_max().alias("issue_idx")
    )
    .collect()
)

In [ ]:
X = train_proc.drop("issue_idx").select(pl.all().exclude(col_targets))
y = train_proc.select(pl.last()).to_numpy().squeeze()

In [ ]:
model = HistGradientBoostingClassifier()
cv = KFold()
cross_validate(model, X, y, cv=cv, scoring = "roc_auc_ovr")  # one vs rest 

In [ ]:
model.fit(X,y)

In [ ]:
preds = model.predict_proba(test)

In [ ]:
preds.shape

In [ ]:
ss = subm.select("id", *[pl.Series(preds[:,i]).alias(col) for i,col in enumerate(col_targets)])
ss.write_csv(path/"subm.csv")

In [ ]:
from kaggle import api
api.competition_submit(path/"subm.csv", "routince boost tree", comp)